In [1]:
# Standard Python modules
import os, sys
import glob
import numpy as np
import pandas as pd
import xarray as xr

path_to_data = '/expanse/nfs/cw3e/cwp140/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write

#####################################################
### LANDSLIDE DATES FROM USGS NEWSWORTHY DATABASE ###
#####################################################
fname = path_to_data + 'downloads/SEAK_News_Reported_Landslides.csv'
df = pd.read_csv(fname)
df = df.set_index(pd.to_datetime(df['Day_min']))
## subset to dates between 2000 and 2024
idx = (df.index >= '2000-01-01') & (df.index <= '2024-12-31')
df = df.loc[idx]
news_landslides = df

# Count unique values in a column
unique_count = news_landslides.index.value_counts()

###################################################
### LANDSLIDE DATES FROM NOAA STORM EVENTS DATA ###
###################################################
## glob files in directory since they have weird names
fname_pattern = path_to_data + 'downloads/noaastormevents/StormEvents_details-ftp_*.csv'
fname_lst = glob.glob(fname_pattern, recursive=False)
df_lst = []
for i, fname in enumerate(sorted(fname_lst)):
    print(fname)
    df = pd.read_csv(fname, header=0)
    ## subset to Juneau WFO
    idx = (df.WFO == 'AJK')
    df = df.loc[idx]

    ## debris flow specific
    idx = df['EVENT_TYPE'] == 'Debris Flow'
    deb_flow = df.loc[idx]
    ## filter by keywords
    idx = (df['EVENT_NARRATIVE'].str.contains('mudslide')) | \
          (df['EVENT_NARRATIVE'].str.contains('debris flow')) | \
          (df['EVENT_NARRATIVE'].str.contains('landslide')) | \
          (df['EVENT_NARRATIVE'].str.contains('mass wasting')) | \
          (df['EPISODE_NARRATIVE'].str.contains('mudslide')) | \
          (df['EPISODE_NARRATIVE'].str.contains('debris flow')) | \
          (df['EPISODE_NARRATIVE'].str.contains('landslide')) | \
          (df['EPISODE_NARRATIVE'].str.contains('mass wasting'))
        
    df = df.loc[idx]
    df_lst.append(df)
    df_lst.append(deb_flow)

df = pd.concat(df_lst)
# ## set begin date to index
# Convert 'YearMonth' to datetime and extract year and month
df['YearMonth'] = pd.to_datetime(df['BEGIN_YEARMONTH'], format='%Y%m')
df['Year'] = df['YearMonth'].dt.year
df['Month'] = df['YearMonth'].dt.month
df['Day'] = pd.to_datetime(df['BEGIN_DAY'], format='%d').dt.day

# Create datetime column
df['DateTime'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df.set_index(pd.to_datetime(df['DateTime']))
# ## subset to start_date and end_date
idx = (df.index >= '2000-01-01') & (df.index <= '2024-12-31')
df = df.loc[idx]

# Count unique values in a column
unique_count2 = df.index.value_counts()

## COMBINE NEWSWORTHY LANDSLIDE DATES WITH NOAA STORM EVENTS DATES
combined_dt_array = pd.concat((unique_count, unique_count2))
combined_dt_array = combined_dt_array.reset_index()

## reduce duplicates
impact_count = combined_dt_array.groupby('index').sum('count')
impact_count = impact_count.sort_values(by='count', ascending=False)
impact_count = impact_count.reset_index()
impact_count

/scratch/dnash/job_39642111/ipykernel_3193093/3343151262.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df.set_index(pd.to_datetime(df['Day_min']))


/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2000_c20220425.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2001_c20220425.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2002_c20220425.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2003_c20220425.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2004_c20220425.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2005_c20220425.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2006_c20250122.csv


/scratch/dnash/job_39642111/ipykernel_3193093/3343151262.py:34: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fname, header=0)


/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2007_c20240216.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2008_c20240620.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2009_c20231116.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2010_c20220425.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2011_c20230417.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2012_c20221216.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2013_c20230118.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2014_c20231116.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2015_c20240716.csv
/expanse/nfs/cw3e/cwp140/downloads/noaastormevents/StormEvents_details-ftp_v1.0_d2016_c20220719.csv


,index,count
0,2023-11-20,25
1,2020-12-01,23
2,2020-11-01,13
3,2015-08-18,12
4,2020-12-02,10
...,...,...
94,2016-09-16,1
95,2015-11-30,1
96,2015-09-10,1
97,2015-09-01,1


In [2]:
## read unique landslide dates
df = pd.read_csv(path_to_out + 'landslide_dates.csv')

IVT_lst = []
Z0_lst = []
UV_lst = []
AR_index_lst = []

for index, row in df.iterrows():
    ## read csv files from landslide dates
    fdate = row['init_date']
    impact_date = row['impact_date']
    model_name = row['model_name']
    F = row['F']
    fname = '/expanse/nfs/cw3e/cwp140/images_historical/{1}/mclimate_init{0}.csv'.format(fdate, impact_date)
    test = pd.read_csv(fname)
    F_lst = np.arange(6, (len(test)*6)+6, 6)
    test['F'] = F_lst
    
    idx = (test['F'] >= F) & (test['F'] < F+24)
    subset = test.loc[idx]
    
    ## pull the maximum values for each var
    IVT_lst.append(subset['IVT'].max())
    Z0_lst.append(subset['Freezing Level'].max())
    UV_lst.append(subset['UV'].max())
    AR_index_lst.append(subset['AR_index'].max())

df['IVT'] = IVT_lst
df['Z0'] = Z0_lst
df['UV'] = UV_lst
df['AR_index'] = AR_index_lst

## select only F=24 for now
idx = (df['F'] == 24)
df = df.loc[idx]
df = df.set_index(pd.to_datetime(pd.to_datetime(df['impact_date'], format="%Y%m%d")))
df = df.drop('impact_date', axis=1)
df = df.reset_index()
df = df.rename(columns={'impact_date': 'index'})
df

,index,init_date,model_name,F,IVT,Z0,UV,AR_index
0,2002-07-10,20020709,GEFSv12_reforecast,24,98.0,75.0,98.0,2
1,2003-10-25,20031024,GEFSv12_reforecast,24,99.0,96.0,99.0,4
2,2005-11-22,20051121,GEFSv12_reforecast,24,99.0,98.0,99.0,4
3,2006-10-04,20061003,GEFSv12_reforecast,24,99.0,75.0,99.0,2
4,2008-11-21,20081120,GEFSv12_reforecast,24,75.0,0.0,98.0,1
...,...,...,...,...,...,...,...,...
93,2005-11-19,20051118,GEFSv12_reforecast,24,99.0,96.0,99.0,3
94,2005-11-20,20051119,GEFSv12_reforecast,24,98.0,75.0,99.0,2
95,2021-08-12,20210811,GEFS_archive,24,100.0,97.0,99.0,3
96,2021-10-01,20210930,GEFS_archive,24,94.0,0.0,99.0,1


In [4]:
merged_df_inner = pd.merge(df, impact_count, on='index')
merged_df_inner

,index,init_date,model_name,F,IVT,Z0,UV,AR_index,count
0,2002-07-10,20020709,GEFSv12_reforecast,24,98.0,75.0,98.0,2,1
1,2003-10-25,20031024,GEFSv12_reforecast,24,99.0,96.0,99.0,4,1
2,2005-11-22,20051121,GEFSv12_reforecast,24,99.0,98.0,99.0,4,3
3,2006-10-04,20061003,GEFSv12_reforecast,24,99.0,75.0,99.0,2,1
4,2008-11-21,20081120,GEFSv12_reforecast,24,75.0,0.0,98.0,1,1
...,...,...,...,...,...,...,...,...,...
92,2005-11-18,20051117,GEFSv12_reforecast,24,99.0,97.0,99.0,3,4
93,2005-11-19,20051118,GEFSv12_reforecast,24,99.0,96.0,99.0,3,2
94,2005-11-20,20051119,GEFSv12_reforecast,24,98.0,75.0,99.0,2,4
95,2021-08-12,20210811,GEFS_archive,24,100.0,97.0,99.0,3,1


In [6]:
tmp = merged_df_inner.sort_values(by='count', ascending=False)
tmp = tmp.rename(columns={"count": "impact_count"})
tmp.to_csv('../out/impact_landslide_table.csv', index=False)

In [10]:
idx = (news_landslides.index >= '2020-11-28') & (news_landslides.index <= '2020-11-30')
news_landslides.loc[idx].Article_link_1

Day_min
2020-11-30    https://petersburg.advantage-preservation.com/...
2020-11-29    https://www.kfsk.org/2020/12/09/some-petersbur...
2020-11-29    https://www.kfsk.org/2020/12/09/some-petersbur...
2020-11-29    https://www.kfsk.org/2020/12/09/some-petersbur...
2020-11-28    https://www.krbd.org/2020/12/01/rain-wind-batt...
Name: Article_link_1, dtype: object

In [11]:
news_landslides.loc[idx]

,Art_ID,LS_ID,Day_min,Day_max,Day_prc,Time_class,Time_min,Time_max,Time_per_min,Time_per_max,...,Pub_day,News_source,Article_link_1,Article_link_2,Article_link_3,Article_link_4,Article_link_5,Article_link_6,Article_link_7,Comments
Day_min,,,,,,,,,,,,,,,,,,,,,
2020-11-30,28,52,11/30/20,12/1/20,Two days,Not listed,NaN,NaN,NaN,NaN,...,12/7/20,PP,https://petersburg.advantage-preservation.com/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-29,42,68,11/29/20,12/5/20,Weekly,Not listed,NaN,NaN,NaN,NaN,...,12/9/20,KFSK,https://www.kfsk.org/2020/12/09/some-petersbur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-29,42,69,11/29/20,12/5/20,Weekly,Not listed,NaN,NaN,NaN,NaN,...,12/9/20,KFSK,https://www.kfsk.org/2020/12/09/some-petersbur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-29,42,70,11/29/20,12/5/20,Weekly,Not listed,NaN,NaN,NaN,NaN,...,12/9/20,KFSK,https://www.kfsk.org/2020/12/09/some-petersbur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-28,47,81,11/28/20,11/28/20,Daily,Not listed,NaN,NaN,NaN,NaN,...,12/1/20,KRBD,https://www.krbd.org/2020/12/01/rain-wind-batt...,https://www.ktoo.org/2020/12/02/wind-rain-batt...,NaN,NaN,NaN,NaN,NaN,NaN
